In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, MaxPooling2D,BatchNormalization, ELU
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from keras import regularizers

In [3]:
tf.__version__

'2.10.0'

# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`.

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [4]:
df = open("C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/dan_train.csv").read().splitlines()
games = [i.split(',',2)[-1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [5]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0

Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [7]:
def prepare_input(moves):
    x = np.zeros((19, 19, 7))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [8]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 100160, Total Moves: 22853380


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [9]:
import random
x = []
y = []
games = random.sample(games, 1200)
for game in games:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

In [10]:
x.shape

(271099, 19, 19, 7)

In [11]:
y.shape

(271099,)

In [12]:
y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [14]:
def create_model():
    inputs = Input(shape=(19, 19, 7))
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=12, filters=32, padding='same', activation='relu', kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=12, filters=1, padding='same', activation='relu',kernel_regularizer= regularizers.l2(0.003))(outputs)
    outputs = BatchNormalization()(outputs)

    outputs = ELU()(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    
    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [15]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 7)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 32)        11008     
                                                                 
 batch_normalization (BatchN  (None, 19, 19, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 32)        25632     
                                                                 
 batch_normalization_1 (Batc  (None, 19, 19, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 32)        9248  

In [16]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
    
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPU


In [17]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 512,
    epochs = 40,
    validation_data=(x_val, y_val),
)

Epoch 1/40
477/477 [==============================] - 33s 51ms/step - loss: 4.8922 - accuracy: 0.0688 - val_loss: 5.0073 - val_accuracy: 0.0578
Epoch 2/40
477/477 [==============================] - 23s 48ms/step - loss: 4.1555 - accuracy: 0.1535 - val_loss: 4.0522 - val_accuracy: 0.1943
Epoch 3/40
477/477 [==============================] - 23s 49ms/step - loss: 3.7314 - accuracy: 0.2615 - val_loss: 3.7085 - val_accuracy: 0.2894
Epoch 4/40
477/477 [==============================] - 23s 48ms/step - loss: 3.4609 - accuracy: 0.3067 - val_loss: 3.5067 - val_accuracy: 0.2979
Epoch 5/40
477/477 [==============================] - 23s 48ms/step - loss: 3.3300 - accuracy: 0.3225 - val_loss: 3.5281 - val_accuracy: 0.3070
Epoch 6/40
477/477 [==============================] - 23s 48ms/step - loss: 3.2467 - accuracy: 0.3317 - val_loss: 3.3535 - val_accuracy: 0.3178
Epoch 7/40
477/477 [==============================] - 23s 49ms/step - loss: 3.1916 - accuracy: 0.3385 - val_loss: 3.3420 - val_accuracy:

In [ ]:
model.save('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/model_dan_tutorial.h5')

# Testing

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
```

- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

The code block below is to use **dan_test_public.csv** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [ ]:
def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

In [ ]:
df = open('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to PublicUpload.csv
with open('C:/Users/Tidera/Downloads/drive-download-20231017T013717Z-001/PublicUpload.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

344/344 [==============================] - 1s 1ms/step


# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!

In [ ]:
print(prediction_chars)

[['ll' 'kl' 'ml' 'jm' 'in']
 ['jp' 'ko' 'lm' 'ip' 'iq']
 ['ld' 'jf' 'fg' 'ad' 'cl']
 ...
 ['js' 'hj' 'hk' 'lr' 'gk']
 ['gi' 'hh' 'cj' 'hg' 'hd']
 ['im' 'jm' 'fl' 'fm' 'gk']]
